In [1]:
!pip install openai

In [2]:
from openai import OpenAI, AssistantEventHandler
from typing_extensions import override

# Initialize the OpenAI client
client = OpenAI()

ImportError: cannot import name 'OpenAI' from 'openai' (/Users/calamos/opt/anaconda3/lib/python3.8/site-packages/openai/__init__.py)

In [ ]:
# Step 1: Create a new Assistant with File Search Enabled
assistant = client.beta.assistants.create(
    name="Document Query Assistant",
    instructions="You are a helpful assistant. Use your knowledge base to answer questions about the uploaded documents.",
    model="gpt-4o",
    tools=[{"type": "file_search"}],
)

# Step 2: Upload files and add them to a Vector Store
vector_store = client.beta.vector_stores.create(name="Document Storage")
file_paths = ["path/to/your/file1.txt", "path/to/your/file2.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id, files=file_streams
)

# Step 3: Update the assistant to use the new Vector Store
assistant = client.beta.assistants.update(
    assistant_id=assistant.id,
    tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

# Step 4: Create a thread and attach files
message_file = client.files.create(
    file=open("path/to/your/file3.pdf", "rb"), purpose="assistants"
)
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            
            "content": "Ask your question here.",
            "attachments": [
                {"file_id": message_file.id, "tools": [{"type": "file_search"}]}
            ],
        }
    ]
)

# Step 5: Create a run and check the output
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > {text}", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()
